Bez równoległości

In [21]:
import random
import math

def dist(a,b):
    return math.hypot(a[0]-b[0], a[1]-b[1])

def tour_len(tour, D):
    return sum(D[tour[i]][tour[(i+1)%len(tour)]] for i in range(len(tour)))

def make_D(coords):
    n = len(coords)
    D = [[0]*n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            D[i][j] = dist(coords[i], coords[j])
    return D

def ga_tsp(coords, pop_size=20, gens=50, mutation_prob=0.1):
    n = len(coords)
    D = make_D(coords)

    population = [random.sample(range(n), n) for _ in range(pop_size)]
    fitnesses = [tour_len(ind, D) for ind in population]

    best_idx = min(range(pop_size), key=lambda i: fitnesses[i])
    best_tour = population[best_idx][:]
    best_fit = fitnesses[best_idx]

    for gen in range(1, gens+1):
        new_pop = []

        elite_idx = min(range(pop_size), key=lambda i: fitnesses[i])
        new_pop.append(population[elite_idx][:])

        while len(new_pop) < pop_size:
            p1, p2 = random.sample(population, 2)
            a,b = sorted(random.sample(range(n),2))
            child = [-1]*n
            child[a:b+1] = p1[a:b+1]
            ptr = 0
            for x in p2:
                if x not in child:
                    while child[ptr]!=-1:
                        ptr+=1
                    child[ptr]=x
            if random.random() < mutation_prob:
                i,j = random.sample(range(n),2)
                child[i],child[j] = child[j],child[i]
            new_pop.append(child)

        population = new_pop
        fitnesses = [tour_len(ind,D) for ind in population]

        idx = min(range(pop_size), key=lambda i: fitnesses[i])
        if fitnesses[idx] < best_fit:
            best_fit = fitnesses[idx]
            best_tour = population[idx][:]

        if gen % 10 == 0 or gen==1:
            avg = sum(fitnesses)/len(fitnesses)
            print(f"Gen {gen}: best={best_fit:.2f}, avg={avg:.2f}")

    return best_tour, best_fit

random.seed(42)
N = 10
coords = [(random.random()*100, random.random()*100) for _ in range(N)]
tour, length = ga_tsp(coords, pop_size=20, gens=50)
print("\nBest tour length:", length)
print("Best tour:", tour)


Gen 1: best=351.76, avg=478.04
Gen 10: best=351.76, avg=491.09
Gen 20: best=320.58, avg=487.87
Gen 30: best=264.14, avg=479.35
Gen 40: best=264.14, avg=461.29
Gen 50: best=264.14, avg=456.40

Best tour length: 264.13887564142317
Best tour: [3, 9, 0, 4, 1, 6, 5, 8, 2, 7]


Ze zrównolegleniem osobników - zad 1


In [2]:
import random
import math
from multiprocessing import Pool, cpu_count

def dist(a,b):
    return math.hypot(a[0]-b[0], a[1]-b[1])

def tour_len(tour, D):
    return sum(D[tour[i]][tour[(i+1)%len(tour)]] for i in range(len(tour)))

def make_D(coords):
    n = len(coords)
    D = [[0]*n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            D[i][j] = dist(coords[i], coords[j])
    return D

def two_opt_once(tour, D):
    n = len(tour)
    best_delta = 0.0
    best_i, best_j = -1, -1
    for i in range(n-2):
        a,b = tour[i], tour[i+1]
        for j in range(i+2, n if i>0 else n-1):
            c,d = tour[j], tour[(j+1)%n]
            delta = (D[a][c]+D[b][d]) - (D[a][b]+D[c][d])
            if delta < best_delta:
                best_delta = delta
                best_i,best_j = i,j
    if best_delta < 0:
        i,j = best_i+1,best_j
        tour[i:j+1] = reversed(tour[i:j+1])
        return tour, True
    return tour, False

def two_opt(tour_D):
    tour,D = tour_D
    improved = True
    it = 0
    while improved and it<50:
        tour, improved = two_opt_once(tour,D)
        it +=1
    return tour

def ga_tsp_parallel(coords, pop_size=20, gens=50, mutation_prob=0.1):
    n = len(coords)
    D = make_D(coords)
    population = [random.sample(range(n), n) for _ in range(pop_size)]
    fitnesses = [tour_len(ind, D) for ind in population]

    best_idx = min(range(pop_size), key=lambda i: fitnesses[i])
    best_tour = population[best_idx][:]
    best_fit = fitnesses[best_idx]

    pool = Pool(processes=max(1, cpu_count()-1))

    for gen in range(1, gens+1):
        new_pop = []

        elite_idx = min(range(pop_size), key=lambda i: fitnesses[i])
        new_pop.append(population[elite_idx][:])

        while len(new_pop) < pop_size:
            p1, p2 = random.sample(population, 2)
            a,b = sorted(random.sample(range(n),2))
            child = [-1]*n
            child[a:b+1] = p1[a:b+1]
            ptr = 0
            for x in p2:
                if x not in child:
                    while child[ptr]!=-1:
                        ptr+=1
                    child[ptr]=x
            if random.random()<mutation_prob:
                i,j = random.sample(range(n),2)
                child[i],child[j]=child[j],child[i]
            new_pop.append(child)

        population = new_pop

        args = [(ind,D) for ind in population]
        population = pool.map(two_opt, args)

        fitnesses = [tour_len(ind,D) for ind in population]
        idx = min(range(pop_size), key=lambda i: fitnesses[i])
        if fitnesses[idx] < best_fit:
            best_fit = fitnesses[idx]
            best_tour = population[idx][:]

        if gen % 10 == 0 or gen==1:
            avg = sum(fitnesses)/len(fitnesses)
            print(f"Gen {gen}: best={best_fit:.2f}, avg={avg:.2f}")

    pool.close()
    pool.join()
    return best_tour, best_fit

if __name__=="__main__":
    random.seed(42)
    N = 100
    coords = [(random.random()*100, random.random()*100) for _ in range(N)]
    tour, length = ga_tsp_parallel(coords, pop_size=20, gens=50)
    print("\nBest tour length:", length)
    print("Best tour:", tour)


Gen 1: best=1208.90, avg=1313.68
Gen 10: best=807.85, avg=829.84
Gen 20: best=805.31, avg=822.64
Gen 30: best=796.71, avg=815.46
Gen 40: best=796.71, avg=811.69
Gen 50: best=796.71, avg=808.17

Best tour length: 796.7067709135551
Best tour: [46, 61, 60, 51, 7, 27, 31, 73, 86, 97, 54, 16, 41, 66, 57, 49, 99, 64, 32, 15, 10, 2, 90, 18, 14, 19, 85, 30, 75, 48, 43, 38, 12, 36, 55, 93, 87, 69, 91, 3, 82, 9, 20, 0, 98, 4, 65, 33, 44, 13, 84, 11, 25, 34, 39, 1, 79, 21, 74, 23, 77, 22, 6, 62, 47, 88, 67, 53, 29, 94, 5, 8, 45, 80, 50, 68, 63, 35, 26, 96, 81, 71, 28, 83, 52, 58, 78, 17, 70, 76, 89, 92, 95, 42, 72, 56, 59, 40, 24, 37]


Zadanie 2 - równoległość osobników na CPU + równoległość długości tras na GPU

In [3]:
try:
    import torch
    print("PyTorch found. CUDA available:", torch.cuda.is_available())
except Exception as e:
    print("PyTorch nie jest zainstalowany lub wystąpił błąd:", e)
    print("W Colabie powinien być preinstalowany. Upewnij się, że runtime ma GPU.")

import random
import math
import torch
from multiprocessing import Pool, cpu_count

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Using device:", device)

def coords_to_tensor(coords, device=device):
    t = torch.tensor(coords, dtype=torch.float32, device=device)
    return t

def make_distance_matrix_torch(coords_tensor):
    return torch.cdist(coords_tensor, coords_tensor, p=2.0)

def population_to_tensor(population):
    return torch.tensor(population, dtype=torch.long)

def tour_lengths_gpu(pop_tensor, D):
    if pop_tensor.device != D.device:
        pop_tensor = pop_tensor.to(D.device)
    pop_size, N = pop_tensor.shape

    idx_from = pop_tensor
    idx_to = torch.roll(pop_tensor, shifts=-1, dims=1)

    distances = D[idx_from, idx_to]
    lengths = distances.sum(dim=1)
    return lengths

def two_opt_once_cpu(tour, D_cpu):
    n = len(tour)
    best_delta = 0.0
    best_i, best_j = -1, -1
    for i in range(0, n-2):
        a, b = tour[i], tour[i+1]
        for j in range(i+2, n if i>0 else n-1):
            c, d_ = tour[j], tour[(j+1)%n]
            delta = (D_cpu[a][c] + D_cpu[b][d_]) - (D_cpu[a][b] + D_cpu[c][d_])
            if delta < best_delta:
                best_delta = delta
                best_i, best_j = i, j
    if best_delta < 0:
        i, j = best_i+1, best_j
        tour[i:j+1] = list(reversed(tour[i:j+1]))
        return tour, True
    return tour, False

def two_opt_cpu(tour, D_cpu, max_iters=50):
    it = 0
    improved = True
    while improved and it < max_iters:
        tour, improved = two_opt_once_cpu(tour, D_cpu)
        it += 1
    return tour

def two_opt_wrapper(args):
    tour, D_cpu, max_iters = args
    return two_opt_cpu(tour, D_cpu, max_iters)

def ga_tsp_gpu(
    coords,
    pop_size=100,
    gens=200,
    crossover_prob=0.9,
    mutation_prob=0.05,
    tournament_k=3,
    memetic=False,
    memetic_iters=50,
    use_pool=True
):
    coords_t = coords_to_tensor(coords, device=device)
    D = make_distance_matrix_torch(coords_t)
    N = coords_t.shape[0]

    D_cpu = None
    if memetic:
        D_cpu = D.cpu().tolist()

    population = [random.sample(range(N), N) for _ in range(pop_size)]

    pop_t = population_to_tensor(population)
    fitnesses = tour_lengths_gpu(pop_t, D).cpu().numpy().tolist()

    best_idx = int(min(range(pop_size), key=lambda i: fitnesses[i]))
    best_tour = population[best_idx][:]
    best_fit = fitnesses[best_idx]

    pool = None
    if memetic and use_pool:
        pool = Pool(processes=max(1, cpu_count()-1))

    for gen in range(1, gens+1):
        new_population = []

        elite_idx = int(min(range(len(population)), key=lambda i: fitnesses[i]))
        new_population.append(population[elite_idx][:])

        while len(new_population) < pop_size:
            def tournament():
                ids = random.sample(range(len(population)), tournament_k)
                return min(ids, key=lambda i: fitnesses[i])
            p1 = population[tournament()][:]
            p2 = population[tournament()][:]

            if p1 == p2:
                p2 = population[tournament()][:]

            a, b = sorted(random.sample(range(N), 2))
            child = [-1]*N
            child[a:b+1] = p1[a:b+1]
            ptr = 0
            for x in p2:
                if x not in child:
                    while child[ptr] != -1:
                        ptr += 1
                    child[ptr] = x

            if random.random() < mutation_prob:
                i, j = random.sample(range(N), 2)
                child[i], child[j] = child[j], child[i]

            new_population.append(child)

        population = new_population

        if memetic:
            args = [(ind, D_cpu, memetic_iters) for ind in population]
            if pool:
                population = pool.map(two_opt_wrapper, args)
            else:
                population = [two_opt_wrapper(a) for a in args]

        pop_t = population_to_tensor(population)
        lengths = tour_lengths_gpu(pop_t, D)
        fitnesses = lengths.cpu().numpy().tolist()
        gen_best_idx = int(min(range(len(population)), key=lambda i: fitnesses[i]))
        gen_best_fit = fitnesses[gen_best_idx]
        if gen_best_fit < best_fit:
            best_fit = gen_best_fit
            best_tour = population[gen_best_idx][:]

        if gen % max(1, gens//10) == 0 or gen == 1:
            avg = sum(fitnesses) / len(fitnesses)
            print(f"Gen {gen:4d}  best={gen_best_fit:.4f}  avg={avg:.4f}")

    if pool:
        pool.close()
        pool.join()

    return best_tour, best_fit

if __name__ == "__main__":
    random.seed(42)
    N = 100
    pop_size = 200
    gens = 50

    coords = [(random.random()*1000, random.random()*1000) for _ in range(N)]

    print("Start GA (GPU accelerated fitness)...")
    best_tour, best_len = ga_tsp_gpu(
        coords,
        pop_size=pop_size,
        gens=gens,
        crossover_prob=0.9,
        mutation_prob=0.05,
        tournament_k=3,
        memetic=True,
        memetic_iters=30,
        use_pool=True
    )
    print("\nBest length:", best_len)
    print("First 30 nodes of tour:", best_tour[:30])


PyTorch found. CUDA available: True
Using device: cuda
Start GA (GPU accelerated fitness)...
Gen    1  best=18476.6523  avg=20500.9634
Gen    5  best=7937.8652  avg=8237.5986
Gen   10  best=7921.0273  avg=7968.4000
Gen   15  best=7921.0273  avg=7923.3582
Gen   20  best=7921.0273  avg=7921.9995
Gen   25  best=7921.0273  avg=7921.5754
Gen   30  best=7921.0273  avg=7927.3497
Gen   35  best=7921.0273  avg=7922.2217
Gen   40  best=7921.0273  avg=7925.2184
Gen   45  best=7921.0273  avg=7924.4018
Gen   50  best=7921.0273  avg=7921.9158

Best length: 7921.02734375
First 30 nodes of tour: [27, 31, 73, 86, 97, 41, 16, 54, 26, 96, 71, 81, 35, 63, 68, 50, 80, 45, 28, 83, 8, 52, 58, 78, 17, 70, 76, 89, 92, 95]
